<a href="https://colab.research.google.com/github/giuschil/asset-management-etf-python/blob/main/asset_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import yfinance as yf
import pandas as pd

def scarica_dati(ticker, start_date, end_date):
    """
    Scarica i dati di chiusura al primo giorno del mese per un ETF.
    """
    dati = yf.download(ticker, start=start_date, end=end_date)['Close']
    # Filtra i dati per il primo giorno di ogni mese
    dati_mensili = dati.resample('MS').first()
    return dati_mensili



In [22]:
def simula_portafoglio(dati1, dati2, dati3, peso1, peso2, peso3, investimento_iniziale, investimento_mensile):
    """
    Simula un portafoglio con un investimento iniziale e aggiunte mensili per tre ETF.
    """
    # Verifica dei dati di input
    if dati1.empty or dati2.empty or dati3.empty:
        raise ValueError("Uno o più dei dataset forniti sono vuoti.")
    if not dati1.index.equals(dati2.index) or not dati1.index.equals(dati3.index):
        raise ValueError("Gli indici dei dataset forniti non corrispondono.")

    # Suddivisione dell'investimento iniziale
    capitale1 = investimento_iniziale * peso1
    capitale2 = investimento_iniziale * peso2
    capitale3 = investimento_iniziale * peso3

    # Creazione del DataFrame per tracciare la simulazione
    portafoglio = pd.DataFrame(index=dati1.index)
    portafoglio['Prezzo ETF1'] = dati1
    portafoglio['Prezzo ETF2'] = dati2
    portafoglio['Prezzo ETF3'] = dati3
    portafoglio['Quote ETF1'] = 0.0
    portafoglio['Quote ETF2'] = 0.0
    portafoglio['Quote ETF3'] = 0.0
    portafoglio['Valore ETF1'] = 0.0
    portafoglio['Valore ETF2'] = 0.0
    portafoglio['Valore ETF3'] = 0.0
    portafoglio['Valore Totale'] = 0.0

    # Acquisto iniziale
    if len(portafoglio) > 0:
        portafoglio.iloc[0, portafoglio.columns.get_loc('Quote ETF1')] = capitale1 / dati1.iloc[0]
        portafoglio.iloc[0, portafoglio.columns.get_loc('Quote ETF2')] = capitale2 / dati2.iloc[0]
        portafoglio.iloc[0, portafoglio.columns.get_loc('Quote ETF3')] = capitale3 / dati3.iloc[0]
    else:
        raise ValueError("Il DataFrame non contiene dati validi per iniziare la simulazione.")

    # Simulazione mensile
    for i in range(1, len(portafoglio)):
        # Prezzi attuali
        prezzo1 = dati1.iloc[i]
        prezzo2 = dati2.iloc[i]
        prezzo3 = dati3.iloc[i]

        # Investimento mensile
        acquisto1 = (investimento_mensile * peso1) / prezzo1 if prezzo1 > 0 else 0
        acquisto2 = (investimento_mensile * peso2) / prezzo2 if prezzo2 > 0 else 0
        acquisto3 = (investimento_mensile * peso3) / prezzo3 if prezzo3 > 0 else 0

        # Aggiungi le quote acquistate
        portafoglio.iloc[i, portafoglio.columns.get_loc('Quote ETF1')] = (
            portafoglio.iloc[i - 1, portafoglio.columns.get_loc('Quote ETF1')] + acquisto1
        )
        portafoglio.iloc[i, portafoglio.columns.get_loc('Quote ETF2')] = (
            portafoglio.iloc[i - 1, portafoglio.columns.get_loc('Quote ETF2')] + acquisto2
        )
        portafoglio.iloc[i, portafoglio.columns.get_loc('Quote ETF3')] = (
            portafoglio.iloc[i - 1, portafoglio.columns.get_loc('Quote ETF3')] + acquisto3
        )

        # Calcola il valore degli ETF
        portafoglio.iloc[i, portafoglio.columns.get_loc('Valore ETF1')] = (
            portafoglio.iloc[i, portafoglio.columns.get_loc('Quote ETF1')] * prezzo1
        )
        portafoglio.iloc[i, portafoglio.columns.get_loc('Valore ETF2')] = (
            portafoglio.iloc[i, portafoglio.columns.get_loc('Quote ETF2')] * prezzo2
        )
        portafoglio.iloc[i, portafoglio.columns.get_loc('Valore ETF3')] = (
            portafoglio.iloc[i, portafoglio.columns.get_loc('Quote ETF3')] * prezzo3
        )

        # Calcola il valore totale del portafoglio
        portafoglio.iloc[i, portafoglio.columns.get_loc('Valore Totale')] = (
            portafoglio.iloc[i, portafoglio.columns.get_loc('Valore ETF1')] +
            portafoglio.iloc[i, portafoglio.columns.get_loc('Valore ETF2')] +
            portafoglio.iloc[i, portafoglio.columns.get_loc('Valore ETF3')]
        )

    return portafoglio


# **Scarica dati**

In [24]:


# Parametri di simulazione
ticker1 = "VUSA.MI"  # Vanguard S&P 500 UCITS ETF
ticker2 = "IEMB.MI"  # iShares Core MSCI EM IMI UCITS ETF
ticker3 = "WSML.MI" # l'iShares MSCI World Small Cap UCITS ETF (Acc)

start_date = "2024-01-01"
end_date = "2025-01-01"
peso1 = 0.65
peso2 = 0.25
peso3 = 0.15


investimento_iniziale = 10000  # €10,000
investimento_mensile = 250  # €200 al mese

# Scarica i dati
dati1 = scarica_dati(ticker1, start_date, end_date)
dati2 = scarica_dati(ticker2, start_date, end_date)
dati3 = scarica_dati(ticker3, start_date, end_date)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['WSML.MI']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [26]:
# Simula il portafoglio
portafoglio = simula_portafoglio(dati1, dati2, dati3, peso1, peso2, peso3, investimento_iniziale, investimento_mensile)

# Mostra il risultato finale
print(portafoglio)
#print(f"Valore finale del portafoglio: {portafoglio['Valore Totale'].iloc[-1]:.2f} €")

IndexError: single positional indexer is out-of-bounds